In [2]:
import cv2
from keras.models import load_model
import numpy as np
from pygame import mixer

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# load alarm
mixer.init()
sound = mixer.Sound('alarm.wav')

# haar cascade files are uesd for the detection of face and eye.
face=cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
leye=cv2.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
reye=cv2.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')

# loading model 
model = load_model('models/cnn.h5')

score=0
thicc=2

# categorize whether eyes in open or closed
def prediction(eyes):
    if eyes == ():
        pred = [2]
        return(pred)
    else:
        for (x,y,w,h) in eyes:
            eye= gray[y:y+h,x:x+w] 
            # modifing the eye for making it fit for the model input.
            eye = cv2.resize(eye,(24,24))
            eye = eye/255 # rescaling
            eye = eye.reshape(1,24,24,1) # model expects a 4-dimensional array with shape [n,h,w,c] as an input.
            # feding it into the model and if it return 1 it means eye is open and if its 0 the eye is closed 
            pred = model.predict_classes(eye)
        return(pred)
    
# accessing webcam
cap = cv2.VideoCapture(0)
while(True):
    # reading the video frame by frame and storing it into frame variable and 
    ret, frame = cap.read()
    
    # stores dimensions of the frames
    height,width = frame.shape[:2]

    # converting an BRG (blue, red, green) image into gray image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # faces, left_eye and right_eye will store the coordinates of face and both the eyes
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)
    
    # buliding a black rectangle at the right corner.
    cv2.rectangle(frame, (0,height-50) , (260,height) , (0,0,0) , thickness=cv2.FILLED )
    
    # build a rectangle around the face
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (0,255,0) , 1 )
    for (x,y,w,h) in left_eye:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (0,255,0) , 1 )
    for (x,y,w,h) in right_eye:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (0,255,0) , 1 )
    
    rpred = prediction(right_eye)
    lpred = prediction(left_eye)
    
    # score calculation.
    if(rpred[0]==0 and lpred[0]==0):
        score=min(score+1,20)
        cv2.putText(frame,"Closed",(10,height-20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
        cv2.putText(frame,'Score:'+str(score),(100,height-20),cv2.FONT_HERSHEY_COMPLEX_SMALL , 1,(255,255,255),1,cv2.LINE_AA)
    elif (rpred[0]==2 and lpred[0]==2):
        score = 0
        cv2.rectangle(frame, (10,height-20) , (10,height-20) , (0,0,0) , thickness=cv2.FILLED )
        cv2.putText(frame,"NO ONE IS PRESENT",(10,height-20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
    
    elif (rpred[0]==1 and lpred[0]==1):
        score=max(score-1,0)
        cv2.putText(frame,"Open",(10,height-20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1,cv2.LINE_AA)
        cv2.putText(frame,'Score:'+str(score),(100,height-20),cv2.FONT_HERSHEY_COMPLEX_SMALL , 1,(255,255,255),1,cv2.LINE_AA)
    
    # alarming sound
    if(score>10):
        #person is feeling sleepy so we beep the alarm
        sound.play()
        if score<5: sound.stop()
        #  this add a red boundary.
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),4) 
    # display os frames
    cv2.imshow('frame',frame)
    
    # waits for key to press (press q to stop the execution)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

<ipython-input-3-c170dcb43c24>:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if eyes == ():
E:\anaconda\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
<ipython-input-3-c170dcb43c24>:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if eyes == ():
E:\anaconda\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.